In [1]:
%pylab inline
import pandas as pd

Populating the interactive namespace from numpy and matplotlib


In [2]:
cd ../

/home/chenxupeng/projects/cardiacai


In [3]:
import os
import re

In [5]:
table = pd.read_excel('data/anzhen/table.csv')
table = table.dropna(subset=[u'病案号'])
table.index = table[u'病案号']

In [6]:
table.head()

,姓名,胸片,病案号,性别,年龄,体重,术前诊断,第一诊断,第二诊断,手术日期
病案号,,,,,,,,,,
2742780,支晨骁,1.0,2742780,男,2岁 9月,17.0,VSD,VSD,NaN,2017-03-01
3443379,韩佳磊,1.0,3443379,男,8月 22天,7.0,VSD,VSD,NaN,2017-03-01
3378357,李瑞豪,1.0,3378357,男,19岁 0月,55.0,VSD,VSD,NaN,2017-03-01
3173310,王宇璐,1.0,3173310,女,1岁 0月,9.5,VSD PFO PS,VSD,右室流出道狭窄,2017-03-01
3193871,王明港,1.0,3193871,男,11月 4天,9.2,VSD,VSD,NaN,2017-03-01


In [230]:
f = h5py.File('data/anzhen/datasets/process_320x320.h5', 'r')
image_id_anzhen = f['image_id'][:]
f.close()

In [10]:
image_id_anzhen_abnormal = [a.split('.')[0] for a in os.listdir('data/anzhen/process_512x624')]

In [11]:
patient_ids = np.asarray([a.split('_')[0] for a in image_id_anzhen_abnormal])
positions = np.asarray([a.split('_')[1] for a in image_id_anzhen_abnormal])

In [12]:
# merge duplicated patient ids
group = table.groupby([u'病案号'])
def merge_diagnosis(x):
    x = x.astype('unicode')
    if len(x.unique()) == 1:
        return x[0]
    else:
        return u','.join(x)
diagnosis = group[u'第一诊断'].aggregate(merge_diagnosis)
diagnosis.name = 'diagnosis'

In [14]:
sample_info_anzhen = pd.DataFrame(diagnosis[patient_ids])
sample_info_anzhen['patient_id'] = [a.split('_')[0] for a in image_id_anzhen_abnormal]
sample_info_anzhen['position'] = [a.split('_')[1] for a in image_id_anzhen_abnormal]
sample_info_anzhen['image_id'] = image_id_anzhen_abnormal
sample_info_anzhen['data_source'] = 'anzhen'
sample_info_anzhen.index = image_id_anzhen_abnormal
sample_info_anzhen.index.name = 'image_id'
sample_info_anzhen['has_heart_trace'] = [os.path.exists('data/anzhen/abnormal_Z/heart_traces/%s.jpg.txt'%a) for a in image_id_anzhen_abnormal]

In [15]:
sample_info_anzhen.head()

,diagnosis,patient_id,position,image_id,data_source,has_heart_trace
image_id,,,,,,
0264717_C,NaN,0264717,C,0264717_C,anzhen,False
0264717_Z,NaN,0264717,Z,0264717_Z,anzhen,True
0265969_Z,NaN,0265969,Z,0265969_Z,anzhen,True
0271852_C,NaN,0271852,C,0271852_C,anzhen,False
0271852_Z,NaN,0271852,Z,0271852_Z,anzhen,True


In [16]:
image_id_anzhen_normal = [os.path.splitext(fname)[0] for fname in os.listdir('data/anzhen/normal_Z/addline')]
sample_info_anzhen_normal = pd.DataFrame({'diagnosis': 'normal',
                                         'patient_id': image_id_anzhen_normal,
                                         'image_id': image_id_anzhen_normal})
sample_info_anzhen_normal['data_source'] = 'anzhen'
sample_info_anzhen_normal['position'] = 'Z'
sample_info_anzhen_normal['has_heart_trace'] = [os.path.exists('data/anzhen/abnormal_Z/heart_traces/%s.jpg.txt'%a) for a in image_id_anzhen_normal]
sample_info_anzhen_normal.index = image_id_anzhen_normal
sample_info_anzhen_normal.index.name = 'image_id'
sample_info_anzhen_normal.head()

,diagnosis,image_id,patient_id,data_source,position,has_heart_trace
image_id,,,,,,
ANON20170303010,normal,ANON20170303010,ANON20170303010,anzhen,Z,False
ANON20170303011,normal,ANON20170303011,ANON20170303011,anzhen,Z,False
ANON201703030110,normal,ANON201703030110,ANON201703030110,anzhen,Z,False
ANON2017030301100,normal,ANON2017030301100,ANON2017030301100,anzhen,Z,False
ANON2017030301101,normal,ANON2017030301101,ANON2017030301101,anzhen,Z,False


In [18]:
# indiana normal samples
image_id_indiana = [a.split('.')[0] for a in os.listdir('data/indiana/normal_Z/noline')]
sample_info_indiana = pd.DataFrame({'image_id': image_id_indiana})
pat_image_id = re.compile(r'([0-9]+)[_]+([CZ])')
sample_info_indiana['patient_id'] = [pat_image_id.match(a).group(1) for a in image_id_indiana]
sample_info_indiana['position'] = [pat_image_id.match(a).group(2) for a in image_id_indiana]
sample_info_indiana['diagnosis'] = 'normal'
sample_info_indiana['data_source'] = 'indiana'
sample_info_indiana['has_heart_trace'] = [os.path.exists('data/indiana/normal_Z/heart_traces/%s.jpg.txt'%a) for a in image_id_indiana]
sample_info_indiana.index = image_id_indiana
sample_info_indiana.index.name = 'image_id'
sample_info_indiana.head()

,image_id,patient_id,position,diagnosis,data_source,has_heart_trace
image_id,,,,,,
1005_Z,1005_Z,1005,Z,normal,indiana,True
1006_Z,1006_Z,1006,Z,normal,indiana,True
1008_Z,1008_Z,1008,Z,normal,indiana,True
1009_Z,1009_Z,1009,Z,normal,indiana,True
100_Z,100_Z,100,Z,normal,indiana,True


In [19]:
sample_info = pd.concat([sample_info_anzhen, sample_info_indiana, sample_info_anzhen_normal], axis=0)
print sample_info.shape
sample_info.head()

(7461, 6)


,data_source,diagnosis,has_heart_trace,image_id,patient_id,position
image_id,,,,,,
0264717_C,anzhen,NaN,False,0264717_C,0264717,C
0264717_Z,anzhen,NaN,True,0264717_Z,0264717,Z
0265969_Z,anzhen,NaN,True,0265969_Z,0265969,Z
0271852_C,anzhen,NaN,False,0271852_C,0271852,C
0271852_Z,anzhen,NaN,True,0271852_Z,0271852,Z


In [20]:
sample_info.to_excel('data/sample_info.xlsx')

In [21]:
sample_info.query('(diagnosis == "normal") and (position == "Z")').shape

(1652, 6)

In [22]:
diagnosis_unique, counts = np.unique(sample_info['diagnosis'].values, return_counts=True)
order = np.argsort(-counts)
diagnosis_unique = diagnosis_unique[order]
counts = counts[order]
for a, c in zip(diagnosis_unique, counts)[:10]:
    print(a, c)

(u'VSD', 3550)
('normal', 1652)
(u'ASD', 806)
(u'TOF', 466)
(u'MI', 169)
(u'PDA', 70)
(u'DORV', 68)
(u'CoA', 52)
(u'TAPVC', 45)
(u'TECD', 44)


/home/chenxupeng/apps/anaconda2/lib/python2.7/site-packages/numpy/lib/arraysetops.py:216: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  flag = np.concatenate(([True], aux[1:] != aux[:-1]))
